Hello! This is notebook for "Week 3 Assingment: Segmenting and Clustering Neighborhoods in Toronto" . All 3 parts of assingment for week 3  are in this notebook! :) Thank you!


## Part 1

### Install libraries

In [2]:
! pip install --upgrade pip 
! pip install geopy
! pip install matplotlib
! pip install sklearn
! pip install folium
! pip install lxml
! pip install numpy
! pip install pandas
! pip install bs4
! pip install  geocoder


Requirement already up-to-date: pip in /Users/julija/.conda/envs/Coursera_Capstone/lib/python3.8/site-packages (20.1.1)


In [4]:
import numpy as np 
import pandas as pd 
import geopy 
import requests 
from bs4 import BeautifulSoup 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import geocoder

print('Libraries imported.')

Libraries imported.


### Let's scrap the page 

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html.parser') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')



Page Scrapped.


In [18]:
# Collecting the data

postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1


In [19]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [ ]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighbors['Borough'].unique()),
        neighbors.shape[0]
    )
)